# Generowanie zdjęcia

In [ ]:
import bpy
import tensorflow as tf
from tensorflow.keras.models import load_model
from PIL import Image as PILImage
import numpy as np

def generate_texture(label):
    # Load the trained generator model
    generator = load_model('D:/cgan_working_i_think/CGEN.h5')

    # Set parameters
    num_images = 1  # Generate one image
    latent_dim = 100  # Dimensionality of the input noise vector

    # Prepare noise
    noise = tf.random.normal([num_images, latent_dim])

    # Create labels
    labels = tf.constant([[label]], dtype=tf.int32)  # 0 for stone, 1 for wood

    # Generate image from the noise and label
    generated_images = generator([noise, labels], training=False)
    generated_image = (generated_images[0] * 255).numpy().astype(np.uint8)

    # Save the generated image
    image_path = "D:/saved_img/generated_texture.png"
    PILImage.fromarray(generated_image).save(image_path)

    return image_path


generate_texture(1)


# Dodawanie przycisków

In [ ]:
bl_info = {
    "name": "Texture Panel",
    "author": "Anna Uchronska",
    "version": (0, 0, 1),
    "blender": (2, 80, 0),
    "location": "3D Viewport > Sidebar > My GAN texture panel",
    "description": "GAN new textures",
    "category": "Development",
}

import bpy

class VIEW3D_PT_my_custom_panel(bpy.types.Panel):  # class naming convention ‘CATEGORY_PT_name’

    # where to add the panel in the UI
    bl_space_type = "VIEW_3D"  # 3D Viewport area (find list of values here https://docs.blender.org/api/current/bpy_types_enum_items/space_type_items.html#rna-enum-space-type-items)
    bl_region_type = "UI"  # Sidebar region (find list of values here https://docs.blender.org/api/current/bpy_types_enum_items/region_type_items.html#rna-enum-region-type-items)

    bl_category = "My GAN texture panel"  # found in the Sidebar
    bl_label = "GAN Textures"  # found at the top of the Panel

    def draw(self, context):
        """define the layout of the panel"""
        row = self.layout.row()
        row.operator("mesh.primitive_cube_add", text="Add Cube")
        row = self.layout.row()
        row.operator("mesh.primitive_uv_sphere_add", text="Add Sphere")
        row = self.layout.row()
        row.operator("object.shade_smooth", text="Stone")
        row = self.layout.row()
        row.operator("object.shade_smooth", text="Wood")


def register():
    bpy.utils.register_class(VIEW3D_PT_my_custom_panel)

def unregister():
    bpy.utils.unregister_class(VIEW3D_PT_my_custom_panel)

if __name__ == "__main__":
    register()

# Nakładanie tekstur

In [ ]:
import bpy
import random


def partially_clean_the_scene():
    # select all object in the scene
    bpy.ops.object.select_all(action="SELECT")

    # delete all selected objects in the scene
    bpy.ops.object.delete()

    # make sure we remove data that was connected to the objects we just deleted
    bpy.ops.outliner.orphans_purge(do_local_ids=True, do_linked_ids=True, do_recursive=True)


def create_material(name):
    # create new material
    material = bpy.data.materials.new(name=name)
    # enable creating a material via nodes
    material.use_nodes = True

    # get a reference to the Principled BSDF shader node
    principled_bsdf_node = material.node_tree.nodes["Principled BSDF"]

    # set the base color of the material
    principled_bsdf_node.inputs["Base Color"].default_value = (0.8, 0.120827, 0.0074976, 1)

    # set the metallic value of the material
    principled_bsdf_node.inputs["Metallic"].default_value = 1.0

    # set the roughness value of the material
    principled_bsdf_node.inputs["Roughness"].default_value = random.uniform(0.1, 1.0)

    return material

def create_material_2(name, image_texture_path):
    material = bpy.data.materials.new(name=name)
    material.use_nodes = True

    # Clear default nodes
    material.node_tree.nodes.clear()

    # Create new texture node
    texture_node = material.node_tree.nodes.new('ShaderNodeTexImage')
    texture_node.location = (-200, 300)  # Adjust location as needed

    # Load image texture
    image_texture = bpy.data.images.load(image_texture_path)

    # Set image texture to the node
    texture_node.image = image_texture

    # Create a Principled BSDF node
    principled_bsdf_node = material.node_tree.nodes.new('ShaderNodeBsdfPrincipled')
    principled_bsdf_node.location = (200, 300)  # Adjust location as needed

    # Create a Material Output node
    material_output_node = material.node_tree.nodes.new('ShaderNodeOutputMaterial')
    material_output_node.location = (400, 300)  # Adjust location as needed

    # Link the nodes
    material.node_tree.links.new(texture_node.outputs['Color'], principled_bsdf_node.inputs['Base Color'])
    material.node_tree.links.new(principled_bsdf_node.outputs['BSDF'], material_output_node.inputs['Surface'])

    return material


def add_mesh():
    # create an ico sphere
    bpy.ops.mesh.primitive_ico_sphere_add(subdivisions=5)
    # shade smooth
    bpy.ops.object.shade_smooth()
    # get reference to mesh object
    mesh_obj = bpy.context.active_object

    return mesh_obj


def main():

    partially_clean_the_scene()

    # name = "my_generated_material"
    # material = create_material(name)

     # Provide the path to your image texture
    image_texture_path = "C:/Users/Jakub/Downloads/grass.png"
    name = "my_generated_material"
    material = create_material_2(name, image_texture_path)


    mesh_obj = add_mesh()

    # apply the material to the mesh object
    mesh_obj.data.materials.append(material)


main()

# Połączenie

In [ ]:
bl_info = {
    "name": "Texture Panel",
    "author": "Your Name",
    "version": (0, 0, 2),
    "blender": (2, 80, 0),
    "location": "3D Viewport > Sidebar > My GAN texture panel",
    "description": "GAN new textures",
    "category": "Development",
}

import bpy
import tensorflow as tf
from tensorflow.keras.models import load_model
from PIL import Image as PILImage
import numpy as np

def generate_texture(label):
    # Load the trained generator model
    generator = load_model('D:/cgan_working_i_think/CGEN.h5')

    # Set parameters
    num_images = 1  # Generate one image
    latent_dim = 100  # Dimensionality of the input noise vector

    # Prepare noise
    noise = tf.random.normal([num_images, latent_dim])

    # Create labels
    labels = tf.constant([[label]], dtype=tf.int32)  # 0 for stone, 1 for wood

    # Generate image from the noise and label
    generated_images = generator([noise, labels], training=False)
    generated_image = (generated_images[0] * 255).numpy().astype(np.uint8)

    # Save the generated image
    image_path = "D:/saved_img/generated_texture.png"
    PILImage.fromarray(generated_image).save(image_path)

    return image_path

def create_material(name, image_texture_path):
    material = bpy.data.materials.new(name)
    material.use_nodes = True

    # Clear default nodes
    material.node_tree.nodes.clear()

    # Create new texture node
    texture_node = material.node_tree.nodes.new('ShaderNodeTexImage')
    texture_node.location = (-200, 300)

    # Load image texture
    image_texture = bpy.data.images.load(image_texture_path)

    # Set image texture to the node
    texture_node.image = image_texture

    # Create a Principled BSDF node
    principled_bsdf_node = material.node_tree.nodes.new('ShaderNodeBsdfPrincipled')
    principled_bsdf_node.location = (200, 300)

    # Create a Material Output node
    material_output_node = material.node_tree.nodes.new('ShaderNodeOutputMaterial')
    material_output_node.location = (400, 300)

    # Link the nodes
    material.node_tree.links.new(texture_node.outputs['Color'], principled_bsdf_node.inputs['Base Color'])
    material.node_tree.links.new(principled_bsdf_node.outputs['BSDF'], material_output_node.inputs['Surface'])

    return material

def apply_texture_to_active_object(label):
    image_texture_path = generate_texture(label)
    name = "my_generated_material"
    material = create_material(name, image_texture_path)

    active_obj = bpy.context.active_object
    if active_obj is not None and active_obj.type == 'MESH':
        if len(active_obj.data.materials):
            active_obj.data.materials[0] = material
        else:
            active_obj.data.materials.append(material)

class VIEW3D_PT_my_custom_panel(bpy.types.Panel):
    bl_space_type = "VIEW_3D"
    bl_region_type = "UI"
    bl_category = "My GAN texture panel"
    bl_label = "GAN Texture Generator"

    def draw(self, context):
        layout = self.layout

        row = layout.row()
        row.operator("mesh.primitive_cube_add", text="Add Cube")

        row = layout.row()
        row.operator("mesh.primitive_uv_sphere_add", text="Add Sphere")

        row = layout.row()
        row.operator("wm.generate_stone_texture", text="Stone")

        row = layout.row()
        row.operator("wm.generate_wood_texture", text="Wood")

class WM_OT_generate_stone_texture(bpy.types.Operator):
    bl_idname = "wm.generate_stone_texture"
    bl_label = "Generate Stone Texture"

    def execute(self, context):
        apply_texture_to_active_object(0)
        return {'FINISHED'}

class WM_OT_generate_wood_texture(bpy.types.Operator):
    bl_idname = "wm.generate_wood_texture"
    bl_label = "Generate Wood Texture"

    def execute(self, context):
        apply_texture_to_active_object(1)
        return {'FINISHED'}

def register():
    bpy.utils.register_class(VIEW3D_PT_my_custom_panel)
    bpy.utils.register_class(WM_OT_generate_stone_texture)
    bpy.utils.register_class(WM_OT_generate_wood_texture)

def unregister():
    bpy.utils.unregister_class(VIEW3D_PT_my_custom_panel)
    bpy.utils.unregister_class(WM_OT_generate_stone_texture)
    bpy.utils.unregister_class(WM_OT_generate_wood_texture)

if __name__ == "__main__":
    register()
